In [4]:
import urllib2
response = urllib2.urlopen("http://undergraduate.bulletins.psu.edu/university-course-descriptions/undergraduate/")
html = response.read()
print html

<!doctype html>
<html class="no-js" xml:lang="en" lang="en" dir="ltr">

<head>
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<script>(function(H){H.className=H.className.replace(/\bno-js\b/,'js')})(document.documentElement)</script>
<title>Undergraduate Courses &lt; Penn State University</title>
<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
<link rel="search" type="application/opensearchdescription+xml"
			href="/search/opensearch.xml" title="Catalog" />
<meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1.0" />
<link href="/favicon.ico" rel="shortcut icon" />
<link rel="stylesheet" type="text/css" href="/css/reset.css" />
<link rel="stylesheet" type="text/css" href="/fonts/font-awesome/font-awesome.min.css" />
<link rel="stylesheet" type="text/css" href="/js/lfjs.css" />
<link rel="stylesheet" type="text/css" href="/css/screen.css" media="screen" />
<link rel="stylesheet" type="text/css" href="/css/print.css" media="pr

In [5]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')
print soup.prettify()

<!DOCTYPE doctype html>
<html class="no-js" dir="ltr" lang="en" xml:lang="en">
 <head>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <script>
   (function(H){H.className=H.className.replace(/\bno-js\b/,'js')})(document.documentElement)
  </script>
  <title>
   Undergraduate Courses &lt; Penn State University
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <link href="/search/opensearch.xml" rel="search" title="Catalog" type="application/opensearchdescription+xml"/>
  <meta content="width=device-width, initial-scale=1.0, minimum-scale=1.0" name="viewport"/>
  <link href="/favicon.ico" rel="shortcut icon"/>
  <link href="/css/reset.css" rel="stylesheet" type="text/css"/>
  <link href="/fonts/font-awesome/font-awesome.min.css" rel="stylesheet" type="text/css"/>
  <link href="/js/lfjs.css" rel="stylesheet" type="text/css"/>
  <link href="/css/screen.css" media="screen" rel="stylesheet" type="text/css"/>
  <link href="/css/print.css" media="p

In [9]:
major_list = soup.find_all(id = "/university-course-descriptions/undergraduate/")
print len(major_list)

1


In [15]:
majors = {} # name: url

In [16]:
cnt = 0
for entry in major_list[0].find_all('a'):
    print entry
    major_name = entry.get_text()
    major_url = entry["href"]
    if major_name in majors:
        print "duplicate majors"
        continue
    majors[major_name] = {"url": major_url}
    cnt += 1
    if (cnt+1)%100 == 0:
        print cnt
print cnt

<a href="/university-course-descriptions/undergraduate/acctg/">Accounting (ACCTG)</a>
<a href="/university-course-descriptions/undergraduate/aersp/">Aerospace Engineering (AERSP)</a>
<a href="/university-course-descriptions/undergraduate/afam/">African American Studies (AFAM)</a>
<a href="/university-course-descriptions/undergraduate/aaas/">African and African American Studies (AAAS)</a>
<a href="/university-course-descriptions/undergraduate/afr/">African Studies (AFR)</a>
<a href="/university-course-descriptions/undergraduate/agbm/">Agribusiness Management (AGBM)</a>
<a href="/university-course-descriptions/undergraduate/aee/">Agricultural and Extension Education (AEE)</a>
<a href="/university-course-descriptions/undergraduate/agsc/">Agricultural Science (AGSC)</a>
<a href="/university-course-descriptions/undergraduate/asm/">Agricultural Systems Management (ASM)</a>
<a href="/university-course-descriptions/undergraduate/ag/">Agriculture (AG)</a>
<a href="/university-course-description

<h1> Done obtaining major urls </h1>

In [42]:
url_head = "http://undergraduate.bulletins.psu.edu"
for major_name in majors:
    major_url = majors[major_name]["url"]
    major_html = urllib2.urlopen(url_head + major_url).read()
    soup = BeautifulSoup(major_html, "html.parser")
    course_list = soup.find_all(class_="courseblock")
    majors[major_name]["courses"] = {}
    for course in course_list:
        course_code = course.find_all(class_="course_code")[0].find_all("span")
        course_major = course_code[0].get_text()
        course_num = course_code[1].get_text()

        course_title = course.find_all(class_ = "course_codetitle")[0].get_text()
        course_credits = course.find_all(class_ = "course_credits")[0].get_text()
        course_description_block = course.find_all(class_ = "courseblockdesc")
        if len(course_description_block) == 0:
            course_description = "" # no description
        else:
            course_description = course_description_block[0].p.get_text()
        course_extra = course.find_all(class_ = "noindent courseblockextra")
        course_prereq = ""
        if len(course_extra) > 0:
            extra_info = course_extra[0].p
            extra_info_head = extra_info.find_all("strong")
            if len(extra_info_head) > 0:
                if "Prerequisite:" in extra_info_head[0].get_text():
                    course_prereq = extra_info.get_text()

        course_code = course_major + " " + course_num
        majors[major_name]["courses"][course_code] = {
            "major": course_major,
            "num": course_num,
            "title": course_title,
            "credits": course_credits,
            "description": course_description,
            "prerequisite": course_prereq
        }
                                   
    print "major:", major_name, "num of courses:", len(majors[major_name]["courses"])

major: Korean (KOR) num of courses: 22
major: Recreation, Park, and Tourism Management (RPTM) num of courses: 60
major: Security and Risk Analysis (SRA) num of courses: 28
major: Graphic Design (GD) num of courses: 32
major: Occupational Therapy (OT) num of courses: 27
major: Education (EDUC) num of courses: 69
major: Mining (MNG) num of courses: 15
major: Forestry (FOR) num of courses: 39
major: Engineering (ENGR) num of courses: 65
major: Science (SC) num of courses: 22
major: Criminal Justice (CRIMJ) num of courses: 71
major: Biorenewable Systems (BRS) num of courses: 20
major: Latin (LATIN) num of courses: 18
major: Supply Chain Management (SCM) num of courses: 29
major: Wildlife and Fisheries Science (WFS) num of courses: 33
major: Workforce Education and Development (WFED) num of courses: 35
major: Physical Therapy (PT) num of courses: 30
major: International Agriculture (INTAG) num of courses: 19
major: Nuclear Engineering (NUCE) num of courses: 27
major: Chemical Engineering (C

In [44]:
cnt = 0
for major_name in majors:
    print major_name
    cnt += len(majors[major_name]["courses"])
    print len(majors[major_name]["courses"])
print "total courses", cnt

Korean (KOR)
22
Recreation, Park, and Tourism Management (RPTM)
60
Security and Risk Analysis (SRA)
28
Graphic Design (GD)
32
Occupational Therapy (OT)
27
Education (EDUC)
69
Mining (MNG)
15
Forestry (FOR)
39
Engineering (ENGR)
65
Science (SC)
22
Criminal Justice (CRIMJ)
71
Biorenewable Systems (BRS)
20
Latin (LATIN)
18
Supply Chain Management (SCM)
29
Wildlife and Fisheries Science (WFS)
33
Workforce Education and Development (WFED)
35
Physical Therapy (PT)
30
International Agriculture (INTAG)
19
Nuclear Engineering (NUCE)
27
Chemical Engineering (CHE)
41
Anthropology (ANTH)
88
Ukrainian (UKR)
7
Arts and Architecture Administrative (AAADM)
1
Kinesiology (KINES)
185
Comparative Literature (CMLIT)
84
Finance (FIN)
58
Petroleum and Natural Gas Engineering (PNG)
33
Arts and Architecture (AA)
30
Spanish (SPAN)
76
Swahili (SWA)
9
Humanities and Social Sciences (HSS)
5
Mineral Processing (MNPR)
6
Digital Media, Arts, and Technology (DIGIT)
9
English as a Second Language (ESL)
9
Political Sci

In [161]:
import cPickle
with open("data/courses_crawled_by_major", "w") as df:
    cPickle.dump(majors, df)

In [154]:
import re
# def abbr_major_name(major_name):
#     major_abbr_list = re.findall(r'\([^(^)]*\)', major_name)
#     if len(major_abbr_list)>1:
#         print major_name
#     return major_abbr_list[-1].strip("()")

# major_names = majors.keys()
# major_names_abbr = map(abbr_major_name, major_names)
# course_name_patterns = map(lambda x: x + " *\d+[A-Z]?", major_names_abbr)
# print major_names_abbr

# course_name_pattern = re.compile("|".join(course_name_patterns))

course_name_list = []
for major_name in majors:
    course_name_list += majors[major_name]["courses"].keys()
# course_name_list = map(lambda x: re.sub(' *', '', x), course_name_list) # delete space
def course_name_normalize(course_name):
    cnn = re.sub(' *', '', course_name) # delete space
    cnn = re.sub(r'(\d+)', lambda x: x.group(0).zfill(3), cnn) # all nums come in three digits with zero paddled
    return cnn
course_name_list = map(course_name_normalize, course_name_list)
print "total courses", len(course_name_list)
print course_name_list


course_name_match = re.compile("|".join(course_name_list))

for major_name in majors:
    print major_name
    major_info = majors[major_name]
    course_list = major_info["courses"]
    for course_name in course_list:
        print course_name # , course_list[course]
        course = course_list[course_name]
        prereq = course["prerequisite"]
        # parsing #
        prereq_parsed = re.sub(r'[^\x00-\x7F]', ' ', prereq)
# # #         print repr(prereq_parsed)
        prereq_parsed = re.sub(r' *', '', prereq_parsed)
# #         print repr(prereq_parsed)
        prereq_parsed = prereq_parsed.strip("\t\n ")
#         print repr(prereq_parsed)     
        prereq_parsed = prereq_parsed.lstrip("Prerequisite: ")
#         print repr(prereq_parsed)
#         print course_name_pattern.findall(prereq_parsed)
        prereq_parsed = re.sub(r'(\d+)', lambda x: x.group(0).zfill(3), prereq_parsed)
        course["prereq_courses"] = course_name_match.findall(prereq_parsed)
        print course_name, course["prereq_courses"], course["prerequisite"]

Languages (Less Commonly Taught) (LANG)
[u'KOR', u'RPTM', u'SRA', u'GD', u'OT', u'EDUC', u'MNG', u'FOR', u'ENGR', u'SC', u'CRIMJ', u'BRS', u'LATIN', u'SCM', u'WFS', u'WFED', u'PT', u'INTAG', u'NUCE', u'CHE', u'ANTH', u'UKR', u'AAADM', u'KINES', u'CMLIT', u'FIN', u'PNG', u'AA', u'SPAN', u'SWA', u'HSS', u'MNPR', u'DIGIT', u'ESL', u'PLSC', u'CAM', u'SWENG', u'MUSIC', u'MIS', u'ME', u'FSC', u'LST', u'IET', u'BME', u'EMCH', u'EMET', u'ENTR', u'ANSC', u'HIED', u'ESC', u'CI', u'HIST', u'STRNG', u'CNED', u'BISC', u'BA', u'RLST', u'CMAS', u'SOSC', u'CRIM', u'FORT', u'GAME', u'SUST', u'GER', u'DS', u'WILDL', u'EME', u'CAMS', u'BMB', u'ARTH', u'AAS', u'INTSP', u'ENT', u'IB', u'PSYCH', u'AAAS', u'BE', u'EMSC', u'HRIM', u'EGEE', u'TURF', u'MEDVL', u'MTHED', u'ECON', u'SSET', u'EDSGN', u'MICRB', u'HONOR', u'AET', u'AGECO', u'PPEM', u'LANG', u'CAS', u'ARTSA', u'PERCN', u'KEYBD', u'HORT', u'CHEM', u'AIR', u'FDSC', u'SLAV', u'RM', u'WLED', u'ARCH', u'NUTR', u'CYBER', u'AG', u'RSOC', u'SSED', u'BESC', u

In [156]:
# course name normalization #
for major_name in majors:
    course_info = majors[major_name]["courses"]
    for course_name in course_info.keys():
        course_name_new = course_name_normalize(course_name)
        course_info[course_name_new] = course_info[course_name]
        del course_info[course_name]

In [160]:
# test #
print majors["Physics (PHYS)"]

{'url': u'/university-course-descriptions/undergraduate/phys/', 'courses': {u'PHYS446': {'prereq_courses': [], 'major': u'PHYS', 'description': u'\n\t\t\t\tDiscussion recent research in physics.\n\t\t\t', 'title': u'PHYS 446: The Year in Physics:  A Seminar on the Latest Research', 'credits': u'\n\t\t\t1 Credits\n\t\t', 'num': u'446', 'prerequisite': u'\nPrerequisite:  PHYS\xa0211\n'}, u'PHYS444': {'prereq_courses': [], 'major': u'PHYS', 'description': u'\n\t\t\t\tModern research topics and career opportunities in physics; employment, graduate education, and tailoring the physics curriculum to meet career goals.  PHYS\xa0444 Topics in Contemporary Physics (2) A course required of all Physics majors, designed to be taken in the Spring semester of the junior year. Introduces students to modern research areas in physics at Penn State and elsewhere. Provides background on career choices available with an undergraduate physics degree, including employment opportunities, planning for graduat

In [151]:
string = "abcd1234"
returned = re.findall(r'(\d+)',string)[0]
print repr(returned)
paddled =  returned.zfill(3)
print paddled
print re.sub(r'(\d+)', lambda x: x.group(0).zfill(3), string)

'1234'
1234
abcd1234
